# Test 03a — Line following (straight + curves)

Uses IR sensor array + ControlChassis to follow tape.

- Press stop button / Ctrl+C to stop
- Tune SPEED and STEER_GAIN if it drifts


In [ ]:
# --- Bootstrap: add repo root + common/lib to sys.path ---
from pathlib import Path
import sys

def add_repo_to_path():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            repo_root = p
            common_lib = p / 'common' / 'lib'
            if str(repo_root) not in sys.path:
                sys.path.insert(0, str(repo_root))
            if str(common_lib) not in sys.path:
                sys.path.insert(0, str(common_lib))
            print('Repo root:', repo_root)
            print('Added common/lib:', common_lib)
            return repo_root
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_to_path()

In [ ]:
import time
from fast_sdk.infra_red import InfraredSensors
from fast_sdk.motors import ControlChassis

IR_ADDR = 0x77

ir = InfraredSensors()
ir.address = IR_ADDR
print('IR addr:', hex(ir.address))

ch = ControlChassis()
print('Chassis:', ch)


In [ ]:
# --- Tuning ---
SPEED = 35          # forward speed
STEER_GAIN = 22     # how strongly we turn
LOST_TURN = 18      # turning rate when line is lost
DT = 0.05           # control loop period

# Weights assume sensors are left->right.
# If your robot turns the wrong way, flip the sign (reverse weights).
WEIGHTS = [-3, -2, -1, 1, 2, 3]

def pretty(vals):
    return ''.join('■' if v else '·' for v in vals)

def compute_error(vals):
    # vals: List[bool]
    idxs = [i for i, v in enumerate(vals) if v]
    if not idxs:
        return None
    # weighted average of active sensors
    w = [WEIGHTS[i] for i in idxs]
    return sum(w) / float(len(w))

def stop():
    try:
        ch.stop()
    except Exception:
        # some images use a different stop method
        try:
            ch.set_velocity(0, 0, 0)
        except Exception:
            pass


In [ ]:
print('Line follow starting. Ctrl+C to stop.')
last_error = 0.0
last_seen_side = 0  # -1 left, +1 right

try:
    while True:
        vals = ir.read_sensor_data()  # List[bool]
        err = compute_error(vals)

        if err is None:
            # Lost the line: gently rotate to reacquire
            turn = LOST_TURN * (1 if last_seen_side >= 0 else -1)
            try:
                ch.set_velocity(0, 0, turn)
            except Exception:
                # fallback: slow forward + turn
                ch.set_velocity(15, 0, turn)
            print('LOST', pretty(vals), vals, 'turn', turn)
            time.sleep(DT)
            continue

        # Remember which side the line was on (for lost recovery)
        last_error = float(err)
        last_seen_side = -1 if err < 0 else (1 if err > 0 else last_seen_side)

        # Steering: positive err -> line is to the right -> turn right (negative yaw) OR vice versa
        # If it turns the wrong way on your robot, flip the sign below.
        turn = -STEER_GAIN * err

        ch.set_velocity(SPEED, 0, turn)
        print(pretty(vals), vals, 'err', round(err, 2), 'turn', round(turn, 1))
        time.sleep(DT)

except KeyboardInterrupt:
    print('Stopping…')
finally:
    stop()
    print('Stopped')
